In [ ]:
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np

from functionsFIMS import getAnalysisNumbers
from runDataClass import runData
from polyaClass import myPolya

In [ ]:
if __name__ == '__main__':
    runNos = getAnalysisNumbers()
    simData = runData(runNos[0])
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)

In [ ]:
binWidth=3
simData.plotAvalancheSize(binWidth)
simData.plotAvalancheFits(binWidth)

In [ ]:
thresh = 10
fitPolya = myPolya(24.701, 0.488)
efficiency = fitPolya.calcEfficiencyErrs(thresh, 0.671, 0.095)
print(f'{efficiency[0]:.3f} (+{efficiency[1]:.3f}, {efficiency[2]:.3f})')

In [ ]:
simData.plotAvalanche2D(plotName='TEST')

In [ ]:
def calcIBF(simData):
    """
    """
    allIons = simData.getDataFrame('ionData')
    posIons = allIons[allIons['Ion Charge'] == 1]
    cathIons = posIons[posIons['Final z'] > simData.getRunParameter('Grid Thickness')]

    numCathode = len(cathIons)
    numAvalanche = simData.getRunParameter('Number of Avalanches')
    
    IBF = numCathode/numAvalanche

    print(len(posIons) - len(cathIons))
    print(f'IBF = {IBF}')
    return IBF

calcIBF(simData)
    